## 방문지 Data Cleansing

- 결측치 및 이상치 처리

In [67]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = r'C:\Windows\Fonts\HMFMMUEX.TTC'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

In [25]:
# 방문지 정보 테이블
visit_tbl = pd.read_csv('data/tn_visit_area_info_방문지정보_E.csv')

# 필요 없는 컬럼 제외
visit_tbl = visit_tbl.drop(['ROAD_NM_ADDR','LOTNO_ADDR','X_COORD','Y_COORD','ROAD_NM_CD','LOTNO_CD','POI_ID','POI_NM','RESIDENCE_TIME_MIN','VISIT_CHC_REASON_CD','LODGING_TYPE_CD','SGG_CD'], axis=1)

In [27]:
visit_tbl.head()

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,VISIT_AREA_TYPE_CD,REVISIT_YN,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION
0,2304300001,e_e000004,1,집,2023-04-30,2023-04-30,21,NaN,NaN,NaN,NaN
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30,2,N,4.0,3.0,4.0
2,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30,2,N,4.0,4.0,4.0
3,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30,2,N,4.0,3.0,3.0
4,2304300005,e_e000004,5,집,2023-04-30,2023-05-01,21,NaN,NaN,NaN,NaN


In [ ]:
# 1. 만족도(DGSTFN), 재방문의향(REVISIT_INTENTION), 추천의향(RCMDTN_INTENTION)은 평균값(집 제외)으로 컬럼 추가
# 2. 총여행일수 : visit_order의 시작과 끝의 VISIT_END_YMD - VISIT_START_YMD
# 3. 여행이동횟수 : TRAVEL_ID 기준 로우 카운트(집 제외)
# 4. 재방문비율 : 여행객의 총 방문지수 대비 재방문(Y) 비율

In [70]:
df = visit_tbl.copy()
df['VISIT_START_YMD'] = pd.to_datetime(df['VISIT_START_YMD'])
df['VISIT_END_YMD'] = pd.to_datetime(df['VISIT_END_YMD'])

# 24는 숙소이고, 8곳의 설문이 없는 것이므로 제외하지 않는 것으로 결정
vis_code = [21, 22, 23] 
temp = df[~df['VISIT_AREA_TYPE_CD'].apply(lambda x: x in vis_code)]

# 1번 만족점수평균값 / 재방문점수평균값 / 추천점수평균값
temp1 = temp.groupby('TRAVEL_ID').agg(DGSTFN_AVG=('DGSTFN', 'mean'),REVISIT_AVG=('REVISIT_INTENTION', 'mean'),RCMDTN_AVG=('RCMDTN_INTENTION', 'mean')).reset_index()
# 2번 총여행일수
temp2 = df.groupby('TRAVEL_ID').apply(lambda x: (x['VISIT_END_YMD'].max() - x['VISIT_START_YMD'].min()).days + 1).reset_index(name='TRIP_DAYS')

# 3번 여행이동횟수
temp3 = temp.groupby('TRAVEL_ID')['VISIT_AREA_NM'].count().reset_index(name='MOVE_CNT')

# 4번 재방문비율
temp4 = temp.groupby('TRAVEL_ID')['REVISIT_YN'].apply(lambda x: (x.str.upper() == 'Y').sum() / len(x)).reset_index(name='VISIT_RATE')

In [72]:
# 1, 2, 3, 4번 결과를 travel_id 기준으로 병합
clean_visit_tbl = temp1.reset_index() \
    .merge(temp2, on='TRAVEL_ID', how='left') \
    .merge(temp3, on='TRAVEL_ID', how='left') \
    .merge(temp4, on='TRAVEL_ID', how='left')

In [73]:
clean_visit_tbl

,index,TRAVEL_ID,DGSTFN_AVG,REVISIT_AVG,RCMDTN_AVG,TRIP_DAYS,MOVE_CNT,VISIT_RATE
0,0,e_e000004,4.000000,3.333333,3.666667,2,3,0.000000
1,1,e_e000006,4.750000,4.750000,4.750000,3,8,0.625000
2,2,e_e000009,4.500000,4.500000,4.500000,3,10,0.200000
3,3,e_e000010,4.250000,4.250000,3.916667,3,12,0.833333
4,4,e_e000011,3.900000,3.700000,3.800000,4,10,0.200000
...,...,...,...,...,...,...,...,...
2555,2555,g_g003294,4.444444,4.111111,4.333333,3,9,0.111111
2556,2556,g_g005600,5.000000,5.000000,5.000000,2,1,1.000000
2557,2557,g_g007343,4.000000,3.000000,4.000000,2,2,0.500000
2558,2558,g_g010556,3.833333,4.000000,4.000000,2,6,0.000000


In [ ]:
# 정제 및 가공된 데이터 저장
clean_visit_tbl.to_csv('data/clean_visit_tbl.csv', encoding='utf-8-sig')

In [75]:
# EDA할 데이터 저장
visit_tbl.to_csv('data/eda_visit_tbl.csv', encoding='utf-8-sig')

--- 

## 코드 통합

comment : 다른 테이블과 1:N으로 JOIN 후 여행id 기준으로 가공 진행하거나 혹은 선가공하고 JOIN할 수 있다는 생각에 두가지 케이스로 나누었고, 병합 진행시 선택해주시면 좋을 것 같습니다.

- 여행id, 방문지id (1:N) 로 확인할 경우

In [ ]:
import numpy as np
import pandas as pd 

# 방문지 정보 테이블
visit_tbl = pd.read_csv('data/tn_visit_area_info_방문지정보_E.csv')

# 필요 없는 컬럼 제외
visit_tbl = visit_tbl.drop(['ROAD_NM_ADDR','LOTNO_ADDR','X_COORD','Y_COORD','ROAD_NM_CD','LOTNO_CD','POI_ID','POI_NM','RESIDENCE_TIME_MIN','VISIT_CHC_REASON_CD','LODGING_TYPE_CD','SGG_CD'], axis=1)

- 여행id로 확인할 경우 

In [ ]:
import numpy as np
import pandas as pd 

# 방문지 정보 테이블
visit_tbl = pd.read_csv('data/tn_visit_area_info_방문지정보_E.csv')

# 필요 없는 컬럼 제외
visit_tbl = visit_tbl.drop(['ROAD_NM_ADDR','LOTNO_ADDR','X_COORD','Y_COORD','ROAD_NM_CD','LOTNO_CD','POI_ID','POI_NM','RESIDENCE_TIME_MIN','VISIT_CHC_REASON_CD','LODGING_TYPE_CD','SGG_CD'], axis=1)

df = visit_tbl.copy()
df['VISIT_START_YMD'] = pd.to_datetime(df['VISIT_START_YMD'])
df['VISIT_END_YMD'] = pd.to_datetime(df['VISIT_END_YMD'])

# VIS = {집:21, 친구/친지집:22, 사무실:23}
vis_code = [21, 22, 23] 
temp = df[~df['VISIT_AREA_TYPE_CD'].apply(lambda x: x in vis_code)]

# 1번 만족점수평균값 / 재방문점수평균값 / 추천점수평균값
temp1 = temp.groupby('TRAVEL_ID').agg(DGSTFN_AVG=('DGSTFN', 'mean'),REVISIT_AVG=('REVISIT_INTENTION', 'mean'),RCMDTN_AVG=('RCMDTN_INTENTION', 'mean')).reset_index()

# 2번 총여행일수
temp2 = df.groupby('TRAVEL_ID').apply(lambda x: (x['VISIT_END_YMD'].max() - x['VISIT_START_YMD'].min()).days + 1).reset_index(name='TRIP_DAYS')

# 3번 여행이동횟수
temp3 = temp.groupby('TRAVEL_ID')['VISIT_AREA_NM'].count().reset_index(name='MOVE_CNT')

# 4번 재방문비율
temp4 = temp.groupby('TRAVEL_ID')['REVISIT_YN'].apply(lambda x: (x.str.upper() == 'Y').sum() / len(x)).reset_index(name='VISIT_RATE')

# 1, 2, 3, 4번 결과를 travel_id 기준으로 병합
clean_visit_tbl = temp1.reset_index() \
    .merge(temp2, on='TRAVEL_ID', how='left') \
    .merge(temp3, on='TRAVEL_ID', how='left') \
    .merge(temp4, on='TRAVEL_ID', how='left')

---

## DESC

![tn_visit_area_info_desc](img/TN_VISIT_AREA_INFO_DESC.png)